In [1]:
import torch
import kenlm
#import ctcdecode
import pickle
import numpy as np

from datasets import load_from_disk, load_dataset, load_metric, Dataset, concatenate_datasets

In [11]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-large-xlsr-ger-chris/checkpoint-51000")

In [2]:
alpha = 2.5 # LM Weight
beta = 0.0 # LM Usage Reward
vocab = ['o', 'm', 'ö', 'v', 'p', 'y', 'z', 'f', 'd', 'j', 'i', 't', 'r', 'ä', 'n', 'w', 'h', 'l', 'u', 'a', 'x', 's', 'b', 'c', 'ß', 'ü', 'e', 'g', 'q', ' ', 'k', '_', '_']

word_lm_scorer = ctcdecode.WordKenLMScorer('lm_data/train.arpa', alpha, beta) # use your own kenlm model

decoder = ctcdecode.BeamSearchDecoder(
    vocab,
    num_workers=2,
    beam_width=128,
    scorers=[word_lm_scorer],
    cutoff_prob=np.log(0.000001),
    cutoff_top_n=40
)

found 1gram
found 2gram


In [ ]:
vocab_dict = tokenizer.get_vocab()

sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())
vocab = [x[1].replace("|", " ") if x[1] not in tokenizer.all_special_tokens else "_" for x in sort_vocab]
print(vocab)

In [68]:
print(vocab_dict)
vocab_dict[' '] = vocab_dict['|']
del vocab_dict['|']

{'o': 0, 'm': 1, 'ö': 2, 'v': 3, 'p': 4, 'y': 5, 'z': 6, 'f': 7, 'd': 8, 'j': 9, 'i': 10, 't': 11, 'r': 12, 'ä': 13, 'n': 14, 'w': 15, 'h': 16, 'l': 17, 'u': 18, 'a': 19, 'x': 20, 's': 21, 'b': 22, 'c': 23, 'ß': 24, 'ü': 25, 'e': 26, 'g': 27, 'q': 28, 'k': 30, '|': 29, '[UNK]': 31, '[PAD]': 32}


In [69]:
print(vocab_dict)

{'o': 0, 'm': 1, 'ö': 2, 'v': 3, 'p': 4, 'y': 5, 'z': 6, 'f': 7, 'd': 8, 'j': 9, 'i': 10, 't': 11, 'r': 12, 'ä': 13, 'n': 14, 'w': 15, 'h': 16, 'l': 17, 'u': 18, 'a': 19, 'x': 20, 's': 21, 'b': 22, 'c': 23, 'ß': 24, 'ü': 25, 'e': 26, 'g': 27, 'q': 28, 'k': 30, '[UNK]': 31, '[PAD]': 32, ' ': 29}


In [137]:
from ctcdecode import CTCBeamDecoder

labels = ['o', 'm', 'ö', 'v', 'p', 'y', 'z', 'f', 'd', 'j', 'i', 't', 'r', 'ä', 'n', 'w', 'h', 'l', 'u', 'a', 'x', 's', 'b', 'c', 'ß', 'ü', 'e', 'g', 'q', ' ', 'k', '_', '_']

decoder = CTCBeamDecoder(
    labels,
    model_path='lm_data/train.arpa',
    alpha=1,
    beta=0,
    cutoff_top_n=100,
    cutoff_prob=np.log(0.000001),
    beam_width=128,
    num_processes=16,
    blank_id=32,
    log_probs_input=True
)

In [18]:
print(vocab_dict)

{'o': 0, 'm': 1, 'ö': 2, 'v': 3, 'p': 4, 'y': 5, 'z': 6, 'f': 7, 'd': 8, 'j': 9, 'i': 10, 't': 11, 'r': 12, 'ä': 13, 'n': 14, 'w': 15, 'h': 16, 'l': 17, 'u': 18, 'a': 19, 'x': 20, 's': 21, 'b': 22, 'c': 23, 'ß': 24, 'ü': 25, 'e': 26, 'g': 27, 'q': 28, 'k': 30, '|': 29, '[UNK]': 31, '[PAD]': 32}


In [139]:
beam_results, beam_scores, timesteps, out_len = decoder.decode(torch.tensor([logs]))

inv_map = {v: k for k, v in vocab_dict.items()}

res = ""
for n in beam_results[0][0][:out_len[0][0]]:
    res = res + inv_map[int(n)]
    
print(res)

ihre fotostrecken erschienen mit modemagazin wie der wolgaostseeka


In [3]:
small_chunk = load_from_disk("small_chunk")

In [ ]:
small_chunk = Dataset.from_dict(res51_full_log[0:200])

In [47]:
logs = res51_full_log[2]['lm_raw']
pred_text = res51_full_log[2]['pred_str']
lm_text = res51_full_log[2]['lm_str']
tar_text = res51_full_log[2]['target_text']
print(np.asarray(logs).shape)
print(pred_text)
print(text)
print(tar_text)

(396, 33)
ihre fortestrecken erschienen mit molemagazine wie der wog at das basarwaryclar
ihre forte strecken erschienen mit modemagazin wie der vogtes basar barrique 
ihre fotostrecken erschienen in modemagazinen wie der vogue harpers bazaar und marie claire 


In [5]:
logs = small_chunk[2]['lm_raw']
pred_text = small_chunk[2]['pred_str']
lm_text = small_chunk[2]['lm_str']
tar_text = small_chunk[2]['target_text']
print(np.asarray(logs).shape)
print(pred_text)
print(lm_text)
print(tar_text)

(396, 33)
ihre fortestrecken erschienen mit molemagazine wie der wog at das basarwaryclar
ihre forte strecken erschienen mit modemagazin wie der vogtes basar barrique 
ihre fotostrecken erschienen in modemagazinen wie der vogue harpers bazaar und marie claire 


In [ ]:
def map_to_result(batch):
    
    model.to("cuda")
    input_values = processor(
          batch["speech"], 
          sampling_rate=batch["sampling_rate"], 
          return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    
    batch["lm_raw_2"] = logits.cpu().numpy()
    
    #batch["lm_str"] = decode(logits[0].cpu().numpy())
    
    return batch

In [ ]:
def infer_lm(batch):
    
    batch["lm_str"] = decode(np.asarray(batch["lm_raw"]))
    return batch

In [3]:
from ctcdecode.prefix import State

# Initialize prefixes
prefixes = State(
    scorers=[word_lm_scorer],
    size=128
)

for prefix in prefixes:
    print(prefix)

Prefix("", 0.0, "None", 0.0, -inf)


In [29]:
logs = res51_full_log[2]['lm_raw']
text = res51_full_log[2]['lm_str']
text = res51_full_log[2]['target_text']
print(len(logs))
print(len(text))
print(text)

396
92
ihre fotostrecken erschienen in modemagazinen wie der vogue harpers bazaar und marie claire 


In [30]:
logs = np.asarray(logs)
nT = logs.shape[0]
print(nT)

396


In [31]:
def get_pruned_vocab_indices(log_probs):
    
    cutoff_prob=np.log(0.000001)
    cutoff_top_n=40
    
    """ Return vocab indices of pruned probabilities of a time step. """

    index_to_prob = [(k, log_probs[k]) for k in range(log_probs.shape[0])]
    index_to_prob = sorted(index_to_prob, key=lambda x: x[1], reverse=True)

    if cutoff_top_n < len(index_to_prob):
        index_to_prob = index_to_prob[:cutoff_top_n]

    if cutoff_prob < 1.0:
        filtered = []
        for x in index_to_prob:
            if x[1] >= cutoff_prob:
                filtered.append(x)
        index_to_prob = filtered

    return [x[0] for x in index_to_prob]

In [32]:
print(len(vocab))
for t in range(2):
    print("")
    step_probs = logs[t]
    print(len(step_probs))
    print(step_probs)
    pruned_step_probs = get_pruned_vocab_indices(step_probs)
    print(len(pruned_step_probs))
    print(pruned_step_probs)

33

33
[ -3.14541698  -3.76524782  -6.66179562  -5.16091967  -3.92799187
  -4.91117573  -2.58480668  -4.42430305   0.25773394  -5.30773211
  -1.49122858  -2.32450509  -4.87767696  -5.76897573  -2.33950615
  -3.94859457  -2.17412162  -3.89775801  -2.74408746  -1.11488378
  -5.61254215  -2.56076002  -2.87019825  -3.36453319  -6.88686419
  -6.19530964  -0.64040613  -3.38129091  -6.03537321  -3.85242057
  -3.87596941 -17.70882416  11.8444109 ]
32
[32, 8, 26, 19, 10, 16, 11, 14, 21, 6, 18, 22, 0, 23, 27, 1, 29, 30, 17, 4, 15, 7, 12, 5, 3, 9, 20, 13, 28, 25, 2, 24]

33
[ -3.36899519  -4.43510294  -6.61256313  -6.11390257  -4.59750032
  -4.83982563  -3.82120395  -5.33151674  -1.2708199   -5.74136782
  -1.47957909  -3.00136232  -4.17404079  -6.08344412  -2.56120777
  -4.98186874  -2.53568602  -3.56372023  -3.00162482  -1.67863286
  -6.66186666  -3.39269567  -3.80653071  -3.3572135   -7.5221324
  -6.34197712  -1.40185916  -3.97075295  -6.73809624  -3.15032697
  -4.36318111 -19.58986282  13.2806

In [33]:
result = ""
for t in range(nT):
    step_probs = logs[t]
    max_index = step_probs.argmax()
    symbol = vocab[max_index]
    result = result + symbol

print(result)
    #pruned_step_probs = get_pruned_vocab_indices(step_probs)

__________________________________________________________________________________________________________i_h__ree_ __f__o_r__t__e____s_tr_e_c____ke__n  __e_r_sscch_i_ee_nn_en_  mi___t_ _m_o_____le____m_a____g__a______z__i____nne_ _w_ie_ d_e_r  _w____o______g___ __________a___t_ _d_a__s__ _b__a_____s_a_____r______w__a____r__y_______c__l_a____r__________________________________________________ 


In [ ]:
def infer_lm(batch, word_lm_scorer, vocabulary):
    from ctcdecode.prefix import State
    import numpy as np

    def get_pruned_vocab_indices(log_probs):
        """ Return vocab indices of pruned probabilities of a time step. """

        index_to_prob = [(k, log_probs[k]) for k in range(log_probs.shape[0])]
        index_to_prob = sorted(index_to_prob, key=lambda x: x[1], reverse=True)
        if 40 < len(index_to_prob):
            index_to_prob = index_to_prob[:40]

        if np.log(0.000001) < 1.0:
            filtered = []
            for x in index_to_prob:
                if x[1] >= np.log(0.000001):
                    filtered.append(x)
            index_to_prob = filtered

        return [x[0] for x in index_to_prob]

    def decode(probs):
        # Num time steps
        nT = probs.shape[0]

        # Initialize prefixes
        prefixes = State(
            scorers=[word_lm_scorer],
            size=128
        )

        # Iterate over timesteps
        for t in range(nT):
            step_probs = probs[t]
            pruned_step_probs = get_pruned_vocab_indices(step_probs)

            # Iterate over symbols
            for v in pruned_step_probs:
                symbol = vocabulary[v]
                symbol_prob = step_probs[v]

                # Iterate over prefixes
                for prefix in prefixes:

                    # If there is a blank, we extend the existing prefix
                    if symbol == '_':
                        prefix.add_p_blank(symbol_prob + prefix.score)

                    else:

                        # If the last symbol is repeated
                        # update the existing prefix
                        if symbol == prefix.symbol:
                            p = symbol_prob + prefix.p_non_blank_prev
                            prefix.add_p_non_blank(p)

                        new_prefix = prefixes.get_prefix(prefix, symbol)

                        if new_prefix is not None:
                            p = -np.inf

                            if symbol == prefix.symbol and \
                                    prefix.p_blank_prev > -np.inf:
                                p = prefix.p_blank_prev + symbol_prob

                            elif prefix.symbol != symbol:
                                p = prefix.score + symbol_prob

                            new_prefix.add_p_non_blank(p)

            prefixes.step()

        prefixes.finalize()

        return prefixes.best()

    batch["lm_str"] = decode(np.asarray(batch["lm_raw"]))
    return batch